In [1]:
%load_ext autoreload
%autoreload 2
import mycode.vap as vap

In [2]:
# The goal is to make a warmup-less search handler

In [53]:
# Now let's prepare another application package with the new search chain
from vespa.package import ServicesConfiguration, ApplicationPackage, Schema, Document, Field
from vespa.configuration.services import services, container, document_api, search, chain, searcher, \
    document_processing, content, documents, document, redundancy, handler, binding

doc_schema = Schema(
    name="doc",
    document=Document(
        fields=[Field(name="id", type="int", indexing=["attribute"]),]
    )
)

application_package = ApplicationPackage(
    name='test',
    schema=[doc_schema],
    services_config=ServicesConfiguration(
        application_name="test",
        services_config=services(
            container(
                document_api(),
                document_processing(),
                search(),
                handler(
                    # binding('http://*/foo/'),
                    bundle="warmup-bundle",
                    id='lt.jocas.examples.FooBar'
                    # id="com.yahoo.search.handler.NoWarmupSearchHandler",
                ),
                id="test_container",
                version="1.0"
            ),
            content(
                redundancy(1),
                documents(
                    document(type="doc", mode="index"),
                ),
                id="test_content",
                version="1.0"
            )
        )
    )
)

In [39]:
# Package into a bundle jar
!(cd ../examples/warmup-bundle && mvn -f pom.xml package)
# There should be a file `../examples/warmup-bundle/target/warmup-bundle-0.0.1-deploy.jar`
!ls -alh ../examples/warmup-bundle/target/warmup-bundle-0.0.1-deploy.jar

[INFO] Scanning for projects...
[INFO] 
[INFO] ------------------< lt.jocas.examples:warmup-bundle >-------------------
[INFO] Building warmup-bundle 0.0.1
[INFO]   from pom.xml
[INFO] --------------------------[ container-plugin ]--------------------------
[WARNING] Ignoring incompatible plugin version 4.0.0-beta-1: The plugin org.apache.maven.plugins:maven-resources-plugin:4.0.0-beta-1 requires Maven version 4.0.0-beta-3
[INFO] Latest version of plugin org.apache.maven.plugins:maven-resources-plugin failed compatibility check
[INFO] Looking for compatible RELEASE version of plugin org.apache.maven.plugins:maven-resources-plugin
[INFO] Selected plugin org.apache.maven.plugins:maven-resources-plugin:3.3.1
[WARNING] Ignoring incompatible plugin version 4.0.0-beta-2: The plugin org.apache.maven.plugins:maven-install-plugin:4.0.0-beta-2 requires Maven version 4.0.0-rc-2
[INFO] Latest version of plugin org.apache.maven.plugins:maven-install-plugin failed compatibility check
[INFO] Looking 

In [33]:
from pathlib import Path

# Currently, pyvespa provides no way to add bundle jars nicely into VAP
# So, we need to do a little dance and add it manually.

tmp_vap_folder = '_tmp'
application_package.to_files(Path(tmp_vap_folder))

vap.add_bundles(
    application_root=tmp_vap_folder,
    bundles=['../examples/warmup-bundle/target/warmup-bundle-0.0.1-deploy.jar']
)


In [13]:

from vespa.deployment import VespaDocker

# In case running colima on macos run the following
# !sudo ln -sf $HOME/.colima/default/docker.sock /var/run/docker.sock
vespa_docker = VespaDocker(
    container_image="vespaengine/vespa:8.596.20",
)

client = vespa_docker.deploy_from_disk(
    application_name='warmup',
    application_root=Path(tmp_vap_folder),
)

Waiting for configuration server, 0/60 seconds...
Waiting for configuration server, 5/60 seconds...
Waiting for application to come up, 0/300 seconds.
Waiting for application to come up, 5/300 seconds.
Waiting for application to come up, 10/300 seconds.
Application is up!
Finished deployment.


In [20]:
def compile_deploy():
    a = !(cd ../examples/warmup-bundle && mvn -f pom.xml package)
    tmp_vap_folder = '_tmp'
    application_package.to_files(Path(tmp_vap_folder))
    vap.add_bundles(
        application_root=tmp_vap_folder,
        bundles=['../examples/warmup-bundle/target/warmup-bundle-0.0.1-deploy.jar']
    )
    vap.redeploy_from_disk(
        docker=vespa_docker,
        application_root=tmp_vap_folder
    )

In [54]:
compile_deploy()

Deploy status code: 200
